In [1]:
import snsims

In [2]:
import numpy as np

The class `snsims.Tiling` is an abstract Base class. Therefore, this cannot be instantiated. It must be subclassed, and the set of methods outlined have to be implemented for this to work.

In [4]:
noTile = snsims.Tiling()

TypeError: Can't instantiate abstract class Tiling with abstract methods __init__, area, pointingSequenceForTile, tileIDSequence, tileIDsForSN

In [5]:
class NoTile(snsims.Tiling):
    pass

In [6]:
noTile = NoTile()

TypeError: Can't instantiate abstract class NoTile with abstract methods __init__, area, pointingSequenceForTile, tileIDSequence, tileIDsForSN

The previous failure is due to required methods or properties not being implemented. Here is an example of an implementation that would not throw this error

In [7]:
class MyTile(snsims.Tiling):
    def __init__(self):
        pass
    @property
    def tileIDSequence(self):
        return np.arange(100)
    
    def tileIDsForSN(self, ra, dec):
        x = ra + dec
        y = np.remainder(x, 100.)
        return np.floor(y)
    def area(self, tileID):
        return 1.
    def pointingSequenceForTile(self, tileID, pointings):
        return None
    

In [8]:
myTile = MyTile()